# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.oncvpsp3.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355352499805                   -1.09    7.0    199ms
  2   -9.356747591382       -2.86       -1.42    1.0   93.1ms
  3   -9.356978953955       -3.64       -2.72    1.9   97.3ms
  4   -9.357012066788       -4.48       -3.12    8.6    285ms
  5   -9.357012364541       -6.53       -3.22    2.6    370ms
  6   -9.357012459436       -7.02       -3.35    1.0   82.6ms
  7   -9.357012508891       -7.31       -3.49    1.0   82.5ms
  8   -9.357012533013       -7.62       -3.66    1.0   81.0ms
  9   -9.357012544921       -7.92       -3.95    1.4   90.0ms
 10   -9.357012547773       -8.54       -4.33    1.4   91.7ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.75256648222                   -0.97    5.6    544ms
  2   -18.79104006894       -1.41       -1.32    1.6    242ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      3
│     10
│      3
│     10
│      1
│     13
│      1
│      2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
  3   -18.78089450340   +   -1.99       -1.97    5.4    430ms
  4   -18.79238420173       -1.94       -2.10    1.8    302ms
  5   -18.79240006739       -4.80       -2.25    1.0    241ms
  6   -18.79240566194       -5.25       -2.66    1.0    240ms
  7   -18.79240744378       -5.75       -2.76    4.1    374ms
  8   -18.79240777814       -6.48       -2.95    1.0    221ms
  9   -18.79240805991       -6.55       -3.02    1.1    231ms
 10   -18.79240844783       -6.41       -3.33    1.1    231ms
 11   -18.79240856777       -6.92    

In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54563985922                   -0.84    9.0    2.27s
  2   -37.56040063120       -1.83       -1.18    1.4    845ms
  3   -37.56177656804       -2.86       -2.59    6.2    1.35s
  4   -37.56452834859       -2.56       -2.40    8.0    1.82s
  5   -37.56453442795       -5.22       -2.44    8.0    1.51s
  6   -37.56455368677       -4.72       -3.39    2.0    852ms
  7   -37.56455393840       -6.60       -3.72    8.1    1.42s
  8   -37.56455405447       -6.94       -4.41    3.4    1.11s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355377422199                   -1.09    6.9    241ms
  2   -9.356783388861       -2.85       -1.91    1.0   72.4ms
  3   -9.356982305564       -3.70       -2.59    4.4    147ms
  4   -9.356994528273       -4.91       -2.62    3.2    155ms
  5   -9.357012167944       -4.75       -3.56    1.0   70.7ms
  6   -9.357012462422       -6.53       -3.86    4.1    389ms
  7   -9.357012544500       -7.09       -4.52    1.8   88.1ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│     10
│      6
│     13
│     14
│     12
│      5
│     13
│      6
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55776099931                   -0.84    9.9    2.36s
  2   -37.52420180129   +   -1.47       -1.53    1.8    754ms
  3   -2.641601089695   +    1.54       -0.29    8.6    2.14s
  4   -37.55980904605        1.54       -1.92    7.5    1.99s
  5   -37.54793127355   +   -1.93       -1.84    2.9    1.45s
  6   -37.38706122631   +   -0.79       -1.44    5.6    1.38s
  7   -37.53891173402       -0.82       -1.81    5.1    1.30s
  8   -37.56336656096       -1.61       -2.34    2.5    890ms
  9   -37.56373393706       -3.43       -2.53    3.8    1.42s
 10   -37.56395184100       -3.66       -2.52    2.5    1.01s
 11   -37.56451992116       -3.25    

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.